In [ ]:
import os
import pandas as pd
import tensorflow as tf
import requests
import json
import base64

In [ ]:
directory_path = '/content/drive/MyDrive/1_movies_per_genre'

# List all files in the directory
files = os.listdir(directory_path)
print("Files in directory:", files)

# Initialize an empty dictionary to hold DataFrames
dataframes = {}

# Iterate over each file in the directory
for file in files:
    # Check if the file is a CSV
    if file.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(directory_path, file)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Store the DataFrame in the dictionary with the filename (without extension) as the key
        dataframes[file[:-4]] = df

        # Optionally, print the first few rows of each DataFrame
        print(f"First few rows of {file}:")
        print(df.head())
        print("\n")

In [ ]:
def prepare_json(text):
    feature_spec = {
        'review': tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, 'utf-8')]))
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]
    
    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [ ]:
def predict(text):
    response = requests.post(
        'http://localhost:8501/v1/models/imdb-review-model:predict',
        data=prepare_json(text)
    )
    
    prediction = response.json()['predictions'][0][0]
    return prediction

In [ ]:
for key, df in dataframes.items():
    text = df['review'][0]  # Adjust the column name if necessary

    # Get prediction for the sample text
    prediction = predict(text)

    # Print the result
    print(f'Review from {key} dataset at index 0:')
    print(text)
    print(f'Positive Review ({prediction})' if prediction > 0.5 else f'Negative Review ({prediction})')
    print("\n")